In [1]:
import numpy as np
import open3d as o3d
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
mesh_box = o3d.geometry.TriangleMesh.create_box(0.4, 0.4, 0.1)
mesh_box.translate([-0.2, -0.2, -0.05])
pcd_box = mesh_box.sample_points_poisson_disk(2048, use_triangle_normal=True)

In [3]:
o3d.visualization.draw_geometries([pcd_box])

In [26]:
# Need to crop negative part of x axis and positive part of y axis
# May need to rotate the object to simulate actual condition
rot_mat = np.array([[0.86, -0.5, 0],[0.49, 0.86, 0], [0, 0, 1]])
pcd_box_rot = copy.deepcopy(pcd_box).rotate(rot_mat)
points = np.asarray(pcd_box_rot.points)
normals = np.asarray(pcd_box_rot.normals)

remove_mask = (points[:,0] < 0.12) * (points[:,1] > -0.12)
keep_mask = ~remove_mask
crop_points = points[keep_mask]
crop_normals = normals[keep_mask]
crop_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(crop_points))
crop_pcd.normals = o3d.utility.Vector3dVector(crop_normals)
crop_pcd = crop_pcd.rotate(rot_mat.T, center=np.array([0, 0, 0]))

In [24]:
o3d.visualization.draw_geometries([crop_pcd])

In [13]:
o3d.io.write_point_cloud("../data/seeds/pointclouds/pose_6_pcd.ply", crop_pcd)

True

In [16]:
# Synthesis 3 seed grasps
def vis_grasp(tip_poses):
    tip_vis_sps = []
    for point in tip_poses:
        sp = o3d.geometry.TriangleMesh.create_sphere(0.01)
        sp.translate(point[:3])
        tip_vis_sps.append(sp)
    return tip_vis_sps

def project_to_pcd(tip_poses, pcd):
    """
    Assume tip_poses is point only, without normals
    """
    kd_tree = o3d.geometry.KDTreeFlann(pcd)
    points = np.asarray(pcd.points)
    normals = np.asarray(pcd.normals)
    new_grasps = np.zeros((tip_poses.shape[0], 6))
    for i, tip_pose in enumerate(tip_poses):
        idx = kd_tree.search_knn_vector_3d(tip_pose, 1)[1]
        new_grasps[i,:3] = points[idx]
        new_grasps[i,3:] = normals[idx]
    return new_grasps

In [28]:
seed_grasp_1 = np.array([[0.14, -0.17, 0.05],
                         [0.2, -0.13, 0.0],
                         [0.13, -0.15, -0.05]])
proj_tip_poses = project_to_pcd(seed_grasp_1, crop_pcd)
vis_sps = vis_grasp(proj_tip_poses)
o3d.visualization.draw_geometries([crop_pcd]+vis_sps)

In [30]:
np.save("../data/seeds/grasps/pose_6.npy", proj_tip_poses.reshape(1, 3, 6))

In [35]:
# Synthesis initial condition bad pointcloud
pcd_box_bad = copy.deepcopy(pcd_box)
aabb = o3d.geometry.AxisAlignedBoundingBox(min_bound=np.array([-0.195, -100, -0.045]),max_bound=np.array([100, 100, 0.045]))
pcd_box_bad = pcd_box_bad.crop(aabb)
o3d.visualization.draw_geometries([pcd_box_bad])

AxisAlignedBoundingBox: min: (-0.2, -0.2, -0.05), max: (0.2, 0.2, 0.05)

In [37]:
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd_box_bad, o3d.utility.DoubleVector([0.01, 0.02]))

In [38]:
o3d.visualization.draw_geometries([mesh])

In [39]:
up_pcd_bad = mesh.sample_points_poisson_disk(1400, use_triangle_normal=True)

In [40]:
o3d.visualization.draw_geometries([up_pcd_bad])

In [41]:
o3d.io.write_point_cloud("../data/seeds/pointclouds/pose_7_pcd.ply", up_pcd_bad)

True